In [1]:
import numpy as np

### Random matrix generation

We need matrix of our quadratic function to be positive definite. The definition is: 

$$ M \text { positive definite } \Longleftrightarrow x^{\top} M x>0 \text { for all } x \in \mathbb{R}^{n} \backslash \mathbf{0} $$

And we know that matrix is p.d. if and only if all of it's eigenvalues are positive. So i have a little function that check if matrix is p.d. Actually we don't really need this, becouse if we create our matrix as $A=A*A^T$ we will definitely get symmetric and p.d. matrix (1-st is obvious, 2-nd is becouse <a href="https://en.wikipedia.org/wiki/Cholesky_decomposition">Cholesky decomposition</a>)

In [2]:
def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

In [3]:
n = 3 # size 

A = np.random.rand(n, n)
A = A@A.T

while not is_pos_def(A):
    A = np.random.rand(n, n)
    A = A@A.T
b = np.random.rand(n, 1)

eps = 1e-6 # precision 

In [4]:
print(A)
print(b)

[[1.49077028 1.47877834 1.2775005 ]
 [1.47877834 1.92387993 1.64779715]
 [1.2775005  1.64779715 1.66452232]]
[[0.89789178]
 [0.38259099]
 [0.07012672]]


$$f(x)=\frac{1}{2} x^{T} A x+x^{T} b$$

We need result to be scalar, but numpy will give as array. So we use np.asscalar

In [5]:
def f(x):
    return np.asscalar(1/2*x.T@A@x + b.T@x)

### Steepest gradient descent

The theory behind this is simple. Let imagine that we have $x^k$ and we want to build $x^{k+1}$ that $f(x^{k+1}) < f(x^k)$. Let $x^{k+1} = x^k+\mu q$. So $q$ is direction, $\mu$ is step size. We can use condition on global minimum to find $\mu$. For our simple and convinient function it sounds like: $\dot{f}(x)=0$. So 
$$\begin{aligned} f\left(x^{k+1}\right) &=f\left(x^{k}+\mu q\right) \equiv \varphi(\mu)=\varphi(0)+\mu \dot{\varphi}(0)+\frac{1}{2} \mu^{2} \ddot{\varphi}(0), \quad \dot{\varphi}=\frac{d \varphi}{d \mu} \end{aligned}$$
Where 
$$\varphi(0)=f\left(x^{k}\right), \dot{\varphi}(0)=q^{T}\left(A x^{k}+b\right), \ddot{\varphi}(0)=q^{T} A q$$
And finally 
$$\bar{\mu}_{k}=-\frac{\dot{\varphi}(0)}{\ddot{\varphi}(0)}=-\frac{q^{T}\left(A x^{k}+b\right)}{q^{T} \cdot A q}$$

We build our ${x^{k+1}}$ using this formulas and get decreasing sequence for $f(x)$. And eventually we get our extrema 

The only thing left is direction of descent. And this is the only part that differ Gradient descent and Coordinate descent. 

In sgd we can take gradient of our function ($\operatorname{grad} f(x)=\left(\frac{\partial f(x)}{\partial x_{1}}, \frac{\partial f(x)}{\partial x_{2}}, \dots \frac{\partial f(x)}{\partial x_{n}}\right)^{T}$), in our case it would be $q=\operatorname{grad} f(x)=A x+b$

In cd we can take $q=e^{i}=(\underbrace{0,0, \ldots, 0,1}_{i}, 0 \ldots, 0)^{T}$ where $e^i$ - i-th unit vector in our vector space

In [6]:
def sgd():
    x = b # init approximation 
    i = 0 # count 

    while True:
        q = A@x + b # grad 

        mu = q.T @ q / (q.T @ A @ q)

        x_old = x
        x = x - np.asscalar(mu) * q

        i += 1
        if((np.linalg.norm(x - x_old) < eps and np.linalg.norm(q) < eps) or i > 1000000):
            break
    return x, i

In [7]:
x_sd, i_sd = sgd()

In [8]:
x_sd, i_sd

(array([[-1.73885235],
        [ 0.20210461],
        [ 1.09234411]]), 42)

In [9]:
f(x_sd)

-0.7036877250494035

### Coordinate descent

E - is utility matrix that allow us to extract unit vectors from it

In [10]:
E = np.eye(n) 
E = np.matrix(E)

In [11]:
def cd():
    x = b
    i = 0

    while True: 
        for j in range(n):
            q = E[j].T
            mu = q.T @ (A@x+ b) / A[j][j]
            
            x_old = x
            x = x - np.asscalar(mu) * q
            
        i += 1
        if((np.linalg.norm(x - x_old) < eps and np.linalg.norm(A@x+b) < eps) or i > 1000000):
            break
    return x, i

In [12]:
x_coord, i_coord = cd()

In [13]:
x_coord, i_coord

(matrix([[-1.73885632],
         [ 0.20210997],
         [ 1.09234154]]), 76)

In [14]:
f(x_coord)

-0.7036877250469962

### Gaussian elimination

Well it is a well known algorithm in linear algebra for soloving systems of linear equations (<a href="https://en.wikipedia.org/wiki/Gaussian_elimination">link</a>)

In [15]:
def gaus(A, b):
    n = A.shape[0]
    for i in range(n):
        b[i] = b[i] / A[i][i]
        A[i] = A[i] / A[i][i]
        for j in range(i + 1, n):
            b[j] = b[j] - A[j][i] * b[i]
            A[j] = A[j] - A[j][i] * A[i]

    for i in range(n - 1, -1, -1):
        for j in range(i - 1, -1, -1):
            b[j] -= b[i] * A[j][i]
    return b

In [16]:
x_pr = gaus(A.copy(), -b.copy())

In [17]:
x_pr

array([[-1.73885285],
       [ 0.20210564],
       [ 1.09234316]])

In [18]:
f(x_pr)

-0.7036877250495972

### Comparison of methods

#### gradient descent

In [19]:
print("Discrepancy with the exact answer: ", np.linalg.norm(x_sd - x_pr))
print("number of steps: ", i_sd)

Discrepancy with the exact answer:  1.489369692730216e-06
number of steps:  42


#### coordinate descent 

In [20]:
print("Discrepancy with the exact answer: ", np.linalg.norm(x_coord - x_pr))
print("number of steps: ", i_coord)

Discrepancy with the exact answer:  5.7788631503617155e-06
number of steps:  76
